<a href="https://colab.research.google.com/github/alisongx/interactive-birthday-atlas/blob/main/Henry's_21st_GEOIDs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import numpy as np

Mounted at /content/drive


In [6]:
hdi = pd.read_csv('/content/drive/MyDrive/hd birthday atlas/[CLEAN] Henrys21stBirthday_12-09_guests.csv')
hdi = hdi.drop(columns=['region'])
hdi['person_name'] = hdi['person_name'].ffill(axis=0)
hdi.loc[140, 'genr_address'] = 'Tunnel Beach, Oregon 97141'
hdi.loc[72, 'genr_address'] = '510 Concord Dr, Menlo Park, CA 94025'

isna = [0, 14, 36, 44, 80]
hd_isna = hdi.loc[isna].copy()
hdi = hdi.drop(isna).reset_index(drop=True)
hdi.iloc[0]

,0
person_name,Amber (one of alison 5)
place,los angeles
address,NaN
place_name,Fred 62
genr_address,"Fred 62, 1850 N Vermont Ave, Los Angeles, CA 9..."
text,fred 62 in la


In [ ]:
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut

hd = hdi.copy()

def is_in_ca(lat, lon):
    CA_NORTH = 42.0
    CA_SOUTH = 32.5
    CA_WEST = -124.4
    CA_EAST = -114.1
    return CA_SOUTH <= lat <= CA_NORTH and CA_WEST <= lon <= CA_EAST

geolocator = Nominatim(user_agent="my_agent")

def geocode_with_retry(address, max_attempts=3):
    address_components = address.split(', ')
    address_combinations = [
        # ', '.join(address_components),
        ', '.join(address_components[:1]),
        ', '.join(address_components[1:]),
    ]

    for addr in address_combinations:
        for attempt in range(max_attempts):
            try:
                location = geolocator.geocode(addr)
                if location:
                    lat, lon = location.latitude, location.longitude
                    if is_in_ca(lat, lon):
                        return lat, lon
                    else:
                        break
                break
            except GeocoderTimedOut:
                if attempt == max_attempts - 1:
                    break
    return np.nan, np.nan

# fill in missing addresses
for i in range(hd.shape[0]):
    if pd.isna(hd['address'].iloc[i]):
        place_name = hd['place_name'].iloc[i] if not pd.isna(hd['place_name'].iloc[i]) else ''
        place = hd['place'].iloc[i] if not pd.isna(hd['place'].iloc[i]) else ''
        hd.loc[i, 'address'] = f"{place_name}, {place}, California".strip(', ')

hd['address'] = hd['address'].str.replace(', ,', ',', regex=False)
hd[['latitude', 'longitude']] = hd['genr_address'].apply(geocode_with_retry).apply(pd.Series)

hd

,person_name,place,address,place_name,genr_address,text,latitude,longitude
0,Amber (one of alison 5),los angeles,"Fred 62, los angeles, California",Fred 62,"Fred 62, 1850 N Vermont Ave, Los Angeles, CA 9...",fred 62 in la,34.104625,-118.291645
1,Amelia Monsour,los angeles,"Blue Ribbon Garden, los angeles, California",Blue Ribbon Garden,"Blue Ribbon Garden, 111 S Grand Ave, Los Angel...",Disney Hall rooftop garden in LA,34.055574,-118.250270
2,Amelia Monsour,Pismo Beach,"Old West Cinnamon Rolls, Pismo Beach, California",Old West Cinnamon Rolls,"Old West Cinnamon Rolls, 861 Dolliver St, Pism...",Old West Cinnamon Rolls in Pismo Beach,35.141585,-120.642431
3,Amelia Monsour,Sierra Madre,"Little Santa Anita, Sierra Madre , California",Little Santa Anita,"Little Santa Anita, Sierra Madre, CA 91024",Little Santa Anita in Sierra Madre (doesn’t lo...,34.155897,-118.029447
4,anna,moraga,"Pinehurst Road, moraga, California",Pinehurst Road,"Pinehurst Road, Moraga, CA 94556","(1) pinehurst road in moraga, ca",34.106034,-118.340468
...,...,...,...,...,...,...,...,...
158,Vika Stukalova,berkeley,"Big C, berkeley, California",Big C,"Big C, Berkeley Fire Trails, Berkeley, CA 94720",the one swing in berk,NaN,NaN
159,wavey cramer,big sur,"Pfeiffer Beach, big sur, California",Pfeiffer Beach,"Pfeiffer Beach, Sycamore Canyon Rd, Big Sur, C...","big sur pink sand key hole beach,",36.239365,-121.817605
160,wavey cramer,guerneville,"Russian River, guerneville, California",Russian River,"Russian River, Guerneville, CA 95446","russian river (guerneville),",38.793469,-123.004315
161,wavey cramer,jenner,"Cafe Aquatica, jenner, California",Cafe Aquatica,"Cafe Aquatica, 10439 CA-1, Jenner, CA 95450",cafe aquatica,38.449039,-123.114884


In [ ]:
loc = geolocator.geocode('4125 Piedmont Ave, Oakland, CA 94611')
print(loc.latitude, loc.longitude)

add = 'Fred 62, 1850 N Vermont Ave, Los Angeles, CA 90027,'
address_components = add.split(', ')
addcombinations = [
        ', '.join(address_components),
        ', '.join(address_components[1:2]),
        ', '.join(address_components[:1]),
        ', '.join(address_components[:2]),
        ', '.join(address_components[1:]),
        ', '.join([address_components[0], address_components[-1]]) if len(address_components) > 1 else address
    ]

addcombinations

hd.loc[37, 'place_name'] = "Emma's House"
hd.loc[68, 'place_name'] = "Henry's Mom's House"
hd.loc[86, 'place_name'] = 'Fort Bragg'
hd.loc[[10, 44], 'place_name'] = 'Carmel-by-the-Sea'

# hd[hd['person_name'] == 'Kyla']

37.82682954545455 -122.25168313636364


,person_name,place,address,place_name,genr_address,text,latitude,longitude
7,antonia,santa cruz,Garden of Eden - Henry Cowell Redwood State Pa...,Garden of Eden - Henry Cowell Redwood State Park,"Garden of Eden, Henry Cowell Redwoods State Pa...",Garden of Eden (classic),NaN,NaN
42,grace,san francisco,"The Interval at Long Now, san francisco, Calif...",The Interval at Long Now,"The Interval at Long Now, Fort Mason Center, L...","the interval, awesome bar in sf.",NaN,NaN
57,Jay,redwood city,"Skyline Trail, redwood city, California",Skyline Trail,"Skyline Trail, Skyline Blvd, Redwood City, CA ...","37.40876° N, 122.29965° W",NaN,NaN
59,JFink,santa cruz,"Waddell Beach, santa cruz, California",Waddell Beach,"Waddell Creek Beach, 3600 Highway 1, Davenport...",Waddell Creek Beach (watch kite surfers!).,NaN,NaN
71,Kai Matson,cayucos,"Estero Bluffs Trail, cayucos, California",Estero Bluffs Trail,"Estero Bluffs Trail, 1 Mile North of Cayucos o...",and Estero bluffs trails in Cayucos,NaN,NaN
84,Kyla,san mateo county,"Martin's Beach, san mateo county, California",Martin's Beach,"Martin's Beach, Martin's Beach Rd, Half Moon B...","martin's beach (this one is top secret, so con...",NaN,NaN
125,Rebekah Murphy,san francisco,"6th Avenue Skate Park, san francisco, California",6th Avenue Skate Park,"6th Avenue Skate Park, 6th Ave & Kirkham St, S...",I really love Golden Gate Park on Sunday and w...,NaN,NaN
127,Rebekah Murphy,redlands,"Caroline Park, redlands, California",Caroline Park,"Caroline Park, 1630 Caroline St, Redlands, CA ...",I have to plug Redlands... sorry. but if you ...,NaN,NaN
135,Shubhangi Bose,"tillamook, OR","Tunnel Beach, tillamook, OR, California",Tunnel Beach,"Tunnel Beach, Oregon 97141","Tunnel Beach (OR),",NaN,NaN
141,Sophia Roy-Burman,trinity county,"Trinity Alps, trinity county, California",Trinity Alps,"Trinity Alps, Trinity Alps Wilderness, Weaverv...",trinity alps!,NaN,NaN


In [ ]:
#shameful manual edits...

hdf = hd.copy()

hdf.loc[7, ['longitude', 'latitude']] = [-122.064625, 37.035238]
hdf.loc[13, ['longitude', 'latitude']] = [-122.008907, 41.240927] #mccloud
hdf.loc[36, ['longitude', 'latitude']] = [-119.628304, 37.730343] #horsetail
hdf.loc[42, ['longitude', 'latitude']] = [-122.427564, 37.804721]
hdf.loc[45, ['longitude', 'latitude']] = [-122.227929, 37.760009] #xmas lane
hdf.loc[57, ['longitude', 'latitude']] = [-122.320605, 37.435343]
hdf.loc[59, ['longitude', 'latitude']] = [-122.274604, 37.096702] #waddell
hdf.loc[68, ['longitude', 'latitude']] = [-122.159183, 37.458628] #ur mom
hdf.loc[71, ['longitude', 'latitude']] = [-120.986634, 35.473485]
hdf.loc[73, ['longitude', 'latitude']] = [-122.209578, 37.827280] #montclair neighborhood
hdf.loc[84, ['longitude', 'latitude']] = [-122.406816, 37.375462] #martin's
hdf.loc[125, ['longitude', 'latitude']] = [-122.464048, 37.772724]
hdf.loc[127, ['longitude', 'latitude']] = [-117.158167, 34.023272] #caro park
hdf.loc[131, ['longitude', 'latitude']] = [-121.498534, 38.583445] #tower theatre
hdf.loc[135, ['longitude', 'latitude']] = [-123.973731, 45.463856] #oregon
hdf.loc[141, ['longitude', 'latitude']] = [-123.048539, 41.005859]
hdf.loc[153, ['longitude', 'latitude']] = [-122.257766, 37.873664] #evans
hdf.loc[158, ['longitude', 'latitude']] = [-122.249686, 37.875483]


In [ ]:
import geopandas as gpd
from shapely.geometry import Point

geometry = [Point(xy) for xy in zip(hdf['longitude'], hdf['latitude'])]
gdf = gpd.GeoDataFrame(hdf, geometry=geometry, crs="EPSG:4326")

In [ ]:
import folium

map = folium.Map(location=[36.7783, -119.4179], zoom_start=6)  # center on California

grouped = gdf.groupby(['latitude', 'longitude'])

location_data = {}
for (lat, lon), group in grouped:
    place_name = group['place_name'].iloc[0]  # use the first place_name as the main header
    entries = group.apply(lambda row: f"{row['person_name']}: {row['text']}", axis=1).tolist()
    location_data[(lat, lon)] = {'place_name': place_name, 'entries': entries}

for (lat, lon), data in location_data.items():
    html = f"""
    <div style="width:300px">
        <h4>{data['place_name']}</h4>
        {'<br>'.join(data['entries'])}
    </div>
    """
    popup = folium.Popup(folium.Html(html, script=True))
    folium.Marker(
        location=[lat, lon],
        popup=popup,
        tooltip=data['place_name']
    ).add_to(map)

map.save("henrys21stgeoids.html")